In [ ]:
import nltk
from re import compile
import string
import io

In [ ]:
with open('corpus.csv', 'rb') as csvfile:
    spamreader = csvfile.readlines()

In [ ]:
corpus = []

In [ ]:
for l in xrange(0,len(spamreader)):
    if spamreader[l].startswith('informatika;') or spamreader[l].startswith('istoriya-istoricheskie-nauki;'):
        a_lines = spamreader[l].split(';')
        art = [a_lines[0], a_lines[1], a_lines[2]]
        corpus.append(art)
    else:
        corpus[-1][2] +=  spamreader[l]
print corpus.__len__()

In [ ]:
del spamreader

hist = [art for art in corpus if art[0] == 'istoriya-istoricheskie-nauki']
info = [art for art in corpus if art[0] == 'informatika']

In [ ]:
def load_ngrams(input_file):
    input_f = io.open(input_file, "r", encoding='utf8')
    n_dict = {}
    for line in input_f:
        line = line.replace('\n', '').split('\t')
        n_dict[tuple(line[0:])] = int(line[0])
    input_f.close()
    return n_dict

def tokenize(corpus):
    news_list = corpus
    words = []
    for text in news_list:
        text_str = ''.join(text[2])
        line = nltk.word_tokenize(text_str)
        line = [word.encode("utf-8").decode("utf-8").lower() for word in line if word not in string.punctuation
                        and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
        words += line
    return words



def calculate_perplexity(category, n3_gram, n2_gram):
    sum_probability = 0
    unknown_words = 0
    min_probability = 1
    sum_log = 0
    
    current_probability= {}
    
    for i in xrange(2, len(category)):
        try:
            current_probability[i] = (n3_gram[(category[i-2], category[i-1], category[i])] + 0.0) 
            / n2_gram[(category[i-2],category[i-1])]
            sum_probability += current_probability
            if current_probability[i] < min_probability:
                min_probability = current_probability[i]
        except KeyError:
            unknown_words +=1
    print (unknown_words, len(category), sum_probability)
    known_words = len(category) - unknown_words
    min_probability /= 10.0
    probability_to_substract = (unknown_words * min_probability + 0.0) / known_words
    for i in xrange(2, len(category)):
        try:
            sum_log += math.log(current_probability[i] - probability_to_substract, 2)
        except KeyError:
            sum_log += math.log(min_probability, 2)

    return sum_log

In [ ]:
def tokenize2(corpus):
    news_list = corpus
    words = []
    r = compile('^[а-яА-Я]+')
    for text in news_list:
        russian = [w for w in filter(r.match, text[2].split(' ')  )]
        lower = [word.decode("utf-8").lower()
                 for word in russian if word not in string.punctuation
                        and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]
                ]
        words += russian
    return words

tokenized_hist =  tokenize2(hist)
tokenized_info =  tokenize2(info)

In [ ]:
gram2 = load_ngrams("2grams-3.txt")
gram3 = load_ngrams("3grams-3.txt")

In [ ]:
perplexity_hist = calculate_perplexity(tokenized_hist, gram2, gram3)
perplexity_info = calculate_perplexity(tokenized_info, gram2, gram3)

history_perplexity = 2**(-perplexity_hist / len(tokenized_hist))
informatics_perplexity = 2**(-perplexity_info / len(tokenized_info))